In [1]:
%load_ext autoreload

In [2]:
#main
%autoreload
import pandas as pd
import numpy as np
import datetime as dt
import scipy as sc
from scipy.stats import norm
import math
import utilities as ut

In [3]:
df=pd.read_csv('EUROSTOXX50_2023_Dataset.csv', index_col=0)

In [4]:
df_ptf=df[824:1592] #we selected 3y from 20-03-2019 bckw.
df_ptf=df_ptf.loc[:,['ADSGn.DE','ALVG.DE','MUVGn.DE','OREP.PA']]
df_den=df[825:1593]
df_den=df_den.loc[:,['ADSGn.DE','ALVG.DE','MUVGn.DE','OREP.PA']]

In [5]:
alpha=0.95
notional=1e7
delta=1
n_asset=4
weights=np.ones((n_asset,1))/n_asset
np_den=df_den.to_numpy()
np_num=df_ptf.to_numpy()

In [6]:
returns=np.log(np_num/np_den)

In [11]:
Loss=-notional*(returns.dot(weights)) #moltiplico per i pesi perchè returns è una matrice num_datexnum_assets e weights è n_assetx1
Loss_mean=np.mean(Loss)
Loss_std=np.std(Loss)
VaR_std=norm.ppf(alpha)
VaR=delta*Loss_mean+math.sqrt(delta)*Loss_std*VaR_std
ES_std=norm.pdf(VaR_std)
ES=delta*Loss_mean+math.sqrt(delta)*Loss_std*ES_std

In [10]:
VaR

nan

In [8]:
VaR, ES=ut.AnalyticalNormalMeasures(alpha, weights, notional, delta, returns)
VaR, ES

(nan, nan)

In [151]:
returns.dot(weights)

array([[-9.73061394e-04],
       [-9.27839361e-03],
       [ 1.38445574e-02],
       [-3.38481992e-03],
       [-1.57144052e-02],
       [ 1.29996926e-02],
       [ 1.47601110e-02],
       [-4.42889396e-03],
       [ 1.55151975e-02],
       [-1.08251061e-02],
       [ 8.72091216e-03],
       [-6.56557626e-03],
       [ 5.23469473e-04],
       [-1.05316341e-02],
       [-2.07936696e-02],
       [-7.59461056e-03],
       [ 4.95193712e-03],
       [-8.96416286e-03],
       [-2.55195518e-02],
       [-4.29343221e-03],
       [ 8.12489355e-03],
       [ 2.56691620e-03],
       [ 1.27669506e-03],
       [ 4.05007922e-03],
       [-5.86637777e-03],
       [-3.11261947e-03],
       [ 3.48744095e-02],
       [-9.22814789e-03],
       [ 6.88501736e-03],
       [ 8.52267432e-03],
       [-7.57685192e-04],
       [ 1.16969869e-02],
       [-8.28749367e-03],
       [ 3.70868809e-03],
       [ 1.08609319e-02],
       [ 7.93628031e-03],
       [-2.13478126e-03],
       [            nan],
       [    